
任务描述：给定原文序列，以及原文的发射矩阵（emissions）;标签类型列表（tags）以及标签之间的转移矩阵（transitions）、开始转移矩阵（start_transitions）和结束转移矩阵（end_transitions）。求使用Viterbi进行Docode后的最佳标注序列。

一、读入原文序列、标签类型列表、发射矩阵、各转移矩阵信息。

二、编写Viterbi算法。

三、使用Viterbi算法进行解码，并输出最佳序列。

四、和最终结果进行对比



----

#### 第一步 : 读取转移矩阵、发射矩阵等信息

In [1]:
import json
import numpy as np

# 标签类型  13 种。
tags = ["O", 
        "B-疾病和诊断", "I-疾病和诊断",
        "B-解剖部位", "I-解剖部位",
        "B-实验室检验", "I-实验室检验",
        "B-影像检查", "I-影像检查",
        "B-手术", "I-手术",
        "B-药物", "I-药物"]
# 原文、观测序列
raw_text = '主 诉 ： 空 腹 、 劳 累 、 精 神 紧 张 后 心 慌 、 出 汗 、 面 色 苍 白 ， B 超 发 现 胰 腺 包 块 周 。 现 病 史 ： 缘 患 者 于 周 前 无 明 显 诱 因 后 出 现 心 慌 、 出 汗 、 面 色 苍 白 ， 伴 呕 吐 、 寒 战 ， 无 腰 背 部 放 散 痛 、 皮 肤 巩 膜 黄 染 、 恶 心 、 高 热 、 腹 泻 、 血 便 、 无 咳 嗽 、 咳 痰 ， 无 胸 闷 、 气 短 、 尿 频 、 尿 急 、 尿 痛 、 血 尿 ， . .'
# 观测序列， 
observation = raw_text.split(' ')

# 注意：所有矩阵的值为log后的概率值，。
start_transitions = [0.1131635531783104,
                    -0.15667293965816498, -0.06260432302951813,
                     -0.057886138558387756, -0.02559879794716835,
                     -0.14357832074165344, -0.010935892350971699,
                     -0.05562453716993332, 0.01728999800980091,
                     -0.0062385560013353825, -0.0031587521079927683,
                     0.027737025171518326, 0.015536400489509106]

end_transitions = [0.056778814643621445,
                   0.00666320463642478, -0.02064545452594757,
                   -0.046536821871995926, 0.03488212823867798,
                   -0.02810691110789776, 0.05207380652427673,
                   -0.13368624448776245, -0.06516466289758682,
                   -0.044199977070093155, -0.1353973001241684,
                   -0.1364254653453827, -0.10228021442890167,]

with open('./data/transitions.txt','r',encoding='utf-8') as rf:
    transitions_str = rf.read()
    transitions = json.loads(transitions_str)
    # 13*13 的标签转移矩阵
    # tranistions[i][j] 表示 tag_i 到 tag_j 的转移分数
    transitions = np.array(transitions)
    
with open('./data/emissions.txt','r',encoding='utf-8') as rf:
    emissions_str = rf.read()
    emissions = json.loads(emissions_str)
    # 13*126 的发射矩阵
    emissions = np.array(emissions)[1:-1]
    emissions = emissions.transpose(1,0)
    
with open('./data/tag_seq.txt','r',encoding='utf-8') as rf:
    tag_seq_str = rf.read()
    tag_seq = tag_seq_str.split(' ')
    # 长 126的标签序列，原文的真实标签序列。
    tag_seq = np.array(tag_seq)

In [100]:
print((transitions.shape))
print((emissions.shape))
print((tag_seq.shape))

(13, 13)
(13, 126)
(126,)


----

#### 第二步：编写Viterbi算法；

In [142]:
# 观测 标签 初始 结束 转移 发射
def viterbi(obs_len, tag_len, start_trans, end_trans, trans, emiss):
    """
    :param obs_len: 观测序列长度 int
    :param tag_len: 隐含序列长度 int
    :param strat_trans:初始概率 list
    :param end_trans:结束概率 list
    :param trans:转移概率矩阵 np.ndarray
    :param emiss:发射概率矩阵 np.ndarray
    :return:最佳路径 np.ndarray
    """
    max_p = np.zeros((tag_len, obs_len))  # max_p每一列为当前观测序列不同tag的最大概率
    path = np.zeros((tag_len, obs_len))  # path每一行存储上max_p对应列的路径
    # taglen = 13
    # obslen = 126


    # 初始化max_p第1个观测节点不同tag的最大概率并初始化path从各个tag出发
    for i in range(tag_len):
        # 都是log之后的值，直接加
        max_p[i][0] = start_trans[i] + emiss[i][0]
        path[i][0] = i
        # 第一个观测点默认路径是自己

    # 遍历第1项后的每一个观测序列，计算其不同tag的最大概率
    for obs_index in range(1, obs_len):
        next_path = np.zeros((tag_len, obs_len)) 
        # 转移到下一个观测点的时候需要添加新path

        # 遍历其每一个tag
        for tag_index in range(tag_len): # 13
            # 根据公式计算累计概率，得到该tag的最大概率
            MAX = -0xffff
            pre_tag = 0
            # 二重循环求概率
            for i in range(tag_len): # 13
                probability = max_p[i][obs_index - 1]  #p(前状态是i的概率) 
                probability += trans[i][tag_index] # p(i转移到tag_index)
                probability += emiss[tag_index][obs_index] #p(当前为tag_index)
                
                
                # 拿前一个点中的最大概率，乘上当前点的转移概率得到最新的概率
                # p(前状态是i的概率) * p(i转移到tag_index) * p(当前为tag_index)
                # 也就是状态转移
                if probability > MAX:
                    # 找出并更新最大概率及其路径
                    MAX = probability
                    pre_tag = i
                    
            # 记录最大概率及路径
            max_p[tag_index][obs_index] = MAX
            for i in range(obs_index):
                next_path[tag_index][i] = path[pre_tag][i]
                # "继承"取到最大概率的隐状态之前的路径（从之前的path中取出某条路径）
            next_path[tag_index][obs_index] = tag_index
        # 更新路径
        path = next_path
    #计算end transition score    
    end_transition_score = []
    for i in range(tag_len):
        end_transition_score.append(end_trans[i] + (max_p[i][obs_len - 1]))
    print(end_transition_score)

        # 结束概率就是计算出来到达最终终点的各个max概率

    # 返回最大概率的路径
    MAX = -0xffff
    pre_tag = 0
    for i in range(tag_len):
        if max_p[i][obs_len - 1] > MAX:
            MAX = max_p[i][obs_len - 1]
            pre_tag = i
    return path[pre_tag]


-----

#### 第三步： 使用Viterbi算法进行解码，并输出最佳序列

In [143]:
obs_len = len(observation)
tag_len = len(tags)

print(obs_len, tag_len)

result = viterbi(obs_len, tag_len, start_transitions,
        end_transitions, transitions, emissions)

string = ""
for i in range(len(result)):
    string += observation[i] + '/' + \
        tags[int(result[i])] + "  "
print(end_transitions)
print(string)

126 13
[2376.0765444880817, 2352.5626549965236, 2353.2450455597136, 2352.5949647149537, 2352.5198804906104, 2351.144967081258, 2350.975611495087, 2351.4168239182327, 2351.2939270308707, 2350.8040599843953, 2351.740050042281, 2350.081648024032, 2350.9328437915538]
[1188.0382722440409, 1176.2813274982618, 1176.6225227798568, 1176.2974823574768, 1176.2599402453052, 1175.572483540629, 1175.4878057475435, 1175.7084119591163, 1175.6469635154353, 1175.4020299921976, 1175.8700250211405, 1175.040824012016, 1175.4664218957769]
主/O  诉/O  ：/O  空/O  腹/B-解剖部位  、/O  劳/O  累/O  、/O  精/O  神/O  紧/O  张/O  后/O  心/B-解剖部位  慌/O  、/O  出/O  汗/O  、/O  面/O  色/O  苍/O  白/O  ，/O  B/B-影像检查  超/I-影像检查  发/O  现/O  胰/B-解剖部位  腺/I-解剖部位  包/O  块/O  周/O  。/O  现/O  病/O  史/O  ：/O  缘/O  患/O  者/O  于/O  周/O  前/O  无/O  明/O  显/O  诱/O  因/O  后/O  出/O  现/O  心/B-解剖部位  慌/O  、/O  出/O  汗/O  、/O  面/O  色/O  苍/O  白/O  ，/O  伴/O  呕/O  吐/O  、/O  寒/O  战/O  ，/O  无/O  腰/B-解剖部位  背/I-解剖部位  部/I-解剖部位  放/O  散/O  痛/O  、/O  皮/B-解剖部位  肤/I-解剖部位  巩/B-解剖部位  膜/

In [140]:

my_seq = []
for i in range(len(result)):
    my_seq.append(tags[int(result[i])])

for i in range(len(my_seq)):
    print(i, tag_seq[i], my_seq[i])


0 O O
1 O O
2 O O
3 O O
4 B-解剖部位 B-解剖部位
5 O O
6 O O
7 O O
8 O O
9 O O
10 O O
11 O O
12 O O
13 O O
14 B-解剖部位 B-解剖部位
15 O O
16 O O
17 O O
18 O O
19 O O
20 O O
21 O O
22 O O
23 O O
24 O O
25 B-影像检查 B-影像检查
26 I-影像检查 I-影像检查
27 O O
28 O O
29 B-解剖部位 B-解剖部位
30 I-解剖部位 I-解剖部位
31 O O
32 O O
33 O O
34 O O
35 O O
36 O O
37 O O
38 O O
39 O O
40 O O
41 O O
42 O O
43 O O
44 O O
45 O O
46 O O
47 O O
48 O O
49 O O
50 O O
51 O O
52 O O
53 B-解剖部位 B-解剖部位
54 O O
55 O O
56 O O
57 O O
58 O O
59 O O
60 O O
61 O O
62 O O
63 O O
64 O O
65 O O
66 O O
67 O O
68 O O
69 O O
70 O O
71 O O
72 B-解剖部位 B-解剖部位
73 I-解剖部位 I-解剖部位
74 I-解剖部位 I-解剖部位
75 O O
76 O O
77 O O
78 O O
79 B-解剖部位 B-解剖部位
80 I-解剖部位 I-解剖部位
81 B-解剖部位 B-解剖部位
82 I-解剖部位 I-解剖部位
83 O O
84 O O
85 O O
86 O O
87 O O
88 O O
89 O O
90 O O
91 O O
92 B-解剖部位 B-解剖部位
93 O O
94 O O
95 O O
96 O O
97 O O
98 O O
99 O O
100 O O
101 O O
102 O O
103 O O
104 O O
105 O O
106 B-解剖部位 B-解剖部位
107 O O
108 O O
109 O O
110 O O
111 O O
112 O O
113 O O
114 O O
115 O O
116 O O
117 O O
118 O 

In [141]:
print(my_seq)
print(tag_seq)

['O', 'O', 'O', 'O', 'B-解剖部位', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-解剖部位', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-影像检查', 'I-影像检查', 'O', 'O', 'B-解剖部位', 'I-解剖部位', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-解剖部位', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-解剖部位', 'I-解剖部位', 'I-解剖部位', 'O', 'O', 'O', 'O', 'B-解剖部位', 'I-解剖部位', 'B-解剖部位', 'I-解剖部位', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-解剖部位', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-解剖部位', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O' 'O' 'O' 'O' 'B-解剖部位' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'B-解剖部位' 'O'
 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'B-影像检查' 'I-影像检查' 'O' 'O' 'B-解剖部位'
 'I-解剖部位' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O'
 'O' 'O' 'O' 'O' 'O' 'O' 'B-解剖部位' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O